In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nyurba,RU,2021-04-24 17:22:13,63.2842,118.3319,31.42,92,71,4.09,broken clouds
1,1,Busselton,AU,2021-04-24 17:20:13,-33.6500,115.3333,53.01,88,1,1.16,clear sky
2,2,Berlevag,NO,2021-04-24 17:22:13,70.8578,29.0864,34.79,75,100,16.44,overcast clouds
3,3,Bluff,NZ,2021-04-24 17:18:34,-46.6000,168.3333,48.99,78,82,8.01,light rain
4,4,Daru,PG,2021-04-24 17:22:13,-9.0763,143.2092,80.04,84,85,5.53,moderate rain


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Daru,PG,2021-04-24 17:22:13,-9.0763,143.2092,80.04,84,85,5.53,moderate rain
6,6,Loja,EC,2021-04-24 17:22:14,-3.9931,-79.2042,84.20,42,40,16.11,light rain
12,12,Ouegoa,NC,2021-04-24 17:22:15,-20.3500,164.4333,76.08,85,92,8.30,moderate rain
20,20,Butaritari,KI,2021-04-24 17:22:17,3.0707,172.7902,82.15,73,99,9.26,light rain
23,23,Buala,SB,2021-04-24 17:22:17,-8.1450,159.5921,79.50,82,54,1.90,broken clouds
27,27,Harper,LR,2021-04-24 17:22:18,4.3750,-7.7169,84.81,73,61,10.78,broken clouds
29,29,Arraial Do Cabo,BR,2021-04-24 17:22:19,-22.9661,-42.0278,78.80,73,20,10.36,few clouds
31,31,Kapaa,US,2021-04-24 17:22:19,22.0752,-159.3190,73.99,73,75,17.27,light rain
35,35,Cidreira,BR,2021-04-24 17:22:21,-30.1811,-50.2056,75.85,62,25,11.54,scattered clouds
37,37,Rikitea,PF,2021-04-24 17:22:21,-23.1203,-134.9692,78.60,71,100,15.43,overcast clouds


In [27]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                178
City                   178
Country                176
Date                   178
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Daru,PG,2021-04-24 17:22:13,-9.0763,143.2092,80.04,84,85,5.53,moderate rain
6,6,Loja,EC,2021-04-24 17:22:14,-3.9931,-79.2042,84.20,42,40,16.11,light rain
12,12,Ouegoa,NC,2021-04-24 17:22:15,-20.3500,164.4333,76.08,85,92,8.30,moderate rain
20,20,Butaritari,KI,2021-04-24 17:22:17,3.0707,172.7902,82.15,73,99,9.26,light rain
23,23,Buala,SB,2021-04-24 17:22:17,-8.1450,159.5921,79.50,82,54,1.90,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
548,548,Bubaque,GW,2021-04-24 17:24:24,11.2833,-15.8333,78.64,68,3,21.16,clear sky
553,553,Navirai,BR,2021-04-24 17:24:25,-23.0650,-54.1906,78.60,38,99,4.32,overcast clouds
556,556,Sabang,ID,2021-04-24 17:24:26,5.8933,95.3214,83.28,71,100,15.88,overcast clouds
557,557,Soyo,AO,2021-04-24 17:23:16,-6.1349,12.3689,80.46,80,100,4.81,light rain


In [41]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Daru,PG,80.04,-9.0763,143.2092,
6,Loja,EC,84.20,-3.9931,-79.2042,
12,Ouegoa,NC,76.08,-20.3500,164.4333,
20,Butaritari,KI,82.15,3.0707,172.7902,
23,Buala,SB,79.50,-8.1450,159.5921,
27,Harper,LR,84.81,4.3750,-7.7169,
29,Arraial Do Cabo,BR,78.80,-22.9661,-42.0278,
31,Kapaa,US,73.99,22.0752,-159.3190,
35,Cidreira,BR,75.85,-30.1811,-50.2056,
37,Rikitea,PF,78.60,-23.1203,-134.9692,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [45]:
# 7. Drop the rows where there is no Hotel Name.
hotels = requests.get(base_url, params=params).json()
hotel_df.dropna()

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Daru,PG,80.04,-9.0763,143.2092,
6,Loja,EC,84.20,-3.9931,-79.2042,
12,Ouegoa,NC,76.08,-20.3500,164.4333,
20,Butaritari,KI,82.15,3.0707,172.7902,
23,Buala,SB,79.50,-8.1450,159.5921,
...,...,...,...,...,...,...
548,Bubaque,GW,78.64,11.2833,-15.8333,
553,Navirai,BR,78.60,-23.0650,-54.1906,
556,Sabang,ID,83.28,5.8933,95.3214,
557,Soyo,AO,80.46,-6.1349,12.3689,


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
marker_layer = gmaps.marker_layer(locations)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))